# Chapter 7. Movie Recommendation System Web Application
The purpose of this chapter is to explain a real case example of the recommendation system in action, using the Django framework. We are going to implement a movie recommendation system in which each user that subscribes to the service will receive suggested movies based on his preferences as we have discussed in Chapter 5, Recommendation systems, also we are going to use the same data which consists of 603 movies rated more than 50 times by 942 users. In order to receive recommendations, each user has to rate a certain number of movies, so an information retrieval system (Chapter 4, Web-mining techniques) to search the movies to rate is implemented. The different parts of the Django application are going to be discussed: settings, models, user login/logout, commands, information retrieval system, recommendation systems, an admin interface and APIs (all the code is available on the GitHub of the author chapter_7 folder at https://github.com/ai2010/machine_learning_for_the_web/tree/master/chapter_7). Since Chapter 6, Basics of Django: a simple web framework just introduced the main features of Django, whenever a new feature is employed a technical explanation is also provided. Now we can start describing the different settings and the initial setup to run the application.

# Application setup

We create and start Django as usual:

>```python
django-admin startproject server_movierecsys  
```

and from the server_movierecsys folder we start the application:

>```python
python manage.py startapp books_recsys_app
```

Now the settings.py needs to be configured. As we see in Chapter 6, Basics of Django: a simple web framework we set the installed apps, HTML templates, a layout formatting folder, and an SQLite database:

```python
INSTALLED_APPS = (
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'rest_framework_swagger',
    'books_recsys_app',
)

TEMPLATE_DIRS = (
    os.path.join(BASE_DIR, 'templates'),
)
STATIC_URL = '/static/'
STATICFILES_DIRS = ( os.path.join(BASE_DIR, "static"), )
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```
Apart from the standard apps, and the rest framework (swagger), the books_recsys_app has been included in the installed apps list.

In this case, we need to load data persistently in the memory so that the user experience is improved by not calculating or retrieving data at each user request. To save data or the results of expensive calculations in the memory, we set up the cache system of Django in settings.py:

```python
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
        'LOCATION': '/var/tmp/django_cache',
        'TIMEOUT': None,
    }
}
```
We have chosen the File Based Cache cache type stored in /var/tmp/django_cache and a None timeout which means the data in the cache will never expire.

To use the admin interface, we set up the superuser account through the command:
```python
python manage.py createsuperuser (admin/admin)
```
The application is live at http://localhost:8000/ by typing:
```python
python manage.py runserver
```

# Models
In this application, we need to store the data related to each movie and the movies' ratings from each user of the website. We set up three models:
```python
class UserProfile(models.Model):
    user = models.ForeignKey(User, unique=True)
    array = jsonfield.JSONField()
    arrayratedmoviesindxs = jsonfield.JSONField()
    lastrecs = jsonfield.JSONField()

    def __unicode__(self):
            return self.user.username

    def save(self, *args, **kwargs):
        create = kwargs.pop('create', None)
        recsvec = kwargs.pop('recsvec', None)
        print 'create:',create
        if create==True:
            super(UserProfile, self).save(*args, **kwargs)
        elif recsvec!=None:
             self.lastrecs = json.dumps(recsvec.tolist())
             super(UserProfile, self).save(*args, **kwargs)
        else:
            nmovies = MovieData.objects.count()
            array = np.zeros(nmovies)
            ratedmovies = self.ratedmovies.all()
            self.arrayratedmoviesindxs = json.dumps([m.movieindx for m in ratedmovies])
            for m in ratedmovies:
                array[m.movieindx] = m.value
            self.array = json.dumps(array.tolist())
            super(UserProfile, self).save(*args, **kwargs)
    
class MovieRated(models.Model):
    user = models.ForeignKey(UserProfile, related_name='ratedmovies')
    movie = models.CharField(max_length=100)
    movieindx = models.IntegerField(default=-1)
    value = models.IntegerField()
    
class MovieData(models.Model):
    title = models.CharField(max_length=100)
    array = jsonfield.JSONField()
    ndim = models.IntegerField(default=300)
    description = models.TextField()
```
The model MovieData stores the data for each movie: title, description, and vector representation (ndim is the dimension of the vector representation). MovieRated records each movie rated by the user logged in (each object MovieRated is associated with has a UserProfile that utilizes the website). The UserProfile model stores all the users that sign up to the website, so they can rate movies and receive recommendations. Each UserProfile extends the default Django user model by adding the array field, which stores all the movie's ratings from the user, and the recsvec field which stores his last recommendations: the save function is overridden to fill the array field with all the MovieRated objects associated with the user (if the else statement is true), and to fill the lastrecs field with the last recommendations (else if statement). Note that the MovieRated model has a UserProfile foreign key with the related_name equal to ratedmovies: in the save function of the UserProfile model, self.ratedmovies.all() refers to all the RatedMovie objects that have the same UserProfile value. The field arrayratedmoviesindxs on the UserProfile model records all the movies rated by the user and it is used by the API of the application.

To write these data structures on the database we need to run:
```python
python manage.py makemigrations
python manage.py migrate
```

# Commands 

The commands used in this application are needed to load the data into the memory (cache) and make the user experience fast. Although the movie database is the same used in Chapter 4, Web mining techniques (that is 603 movies rated more than 50 times by 942 users), each movie needs a description to set up an information retrieval system on the movies to rate. The first command we develop takes all the movie titles in the utility matrix used in Chapter 4, Web Mining Techniques and collects the corresponding descriptions from Open Movie Database (OMDb) online service:
```python
from django.core.management.base import BaseCommand
import os
import optparse
import numpy as np
import json
import pandas as pd
import requests
class Command(BaseCommand):

    option_list = BaseCommand.option_list + (
            optparse.make_option('-i', '--input', dest='umatrixfile',
                                 type='string', action='store',
                                 help=('Input utility matrix')),   
            optparse.make_option('-o', '--outputplots', dest='plotsfile',
                                 type='string', action='store',
                                 help=('output file')),  
            optparse.make_option('--om', '--outputumatrix', dest='umatrixoutfile',
                                 type='string', action='store',
                                 help=('output file')),            
        )
        
        
    def getplotfromomdb(self,col,df_moviesplots,df_movies,df_utilitymatrix):
        string = col.split(';')[0]
        
        title=string[:-6].strip()
        year = string[-5:-1]      
        plot = ' '.join(title.split(' ')).encode('ascii','ignore')+'. '
        
        url = "http://www.omdbapi.com/?t="+title+"&y="+year+"&plot=full&r=json"
        
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36"}
        r = requests.get(url,headers=headers)
        jsondata =  json.loads(r.content)
        if 'Plot' in jsondata:
            #store plot + title
            plot += jsondata['Plot'].encode('ascii','ignore')

        if plot!=None and plot!='' and plot!=np.nan and len(plot)>3:#at least 3 letters to consider the movie
            df_moviesplots.loc[len(df_moviesplots)]=[string,plot]
            df_utilitymatrix[col] = df_movies[col]
            print len(df_utilitymatrix.columns)

        return df_moviesplots,df_utilitymatrix
    
    def handle(self, *args, **options):
        pathutilitymatrix = options['umatrixfile']
        df_movies = pd.read_csv(pathutilitymatrix)
        movieslist = list(df_movies.columns[1:])

        df_moviesplots = pd.DataFrame(columns=['title','plot'])
        df_utilitymatrix = pd.DataFrame()
        df_utilitymatrix['user'] = df_movies['user']

        for m in movieslist[:]:
            df_moviesplots,df_utilitymatrix=self.getplotfromomdb(m,df_moviesplots,df_movies,df_utilitymatrix)

        outputfile = options['plotsfile']
        df_moviesplots.to_csv(outputfile, index=False)
        outumatrixfile = options['umatrixoutfile']
        df_utilitymatrix.to_csv(outumatrixfile, index=False)
```
The command syntax is:
```python
python manage.py --input=utilitymatrix.csv --outputplots=plots.csv –outputumatrix='umatrix.csv'
```
Each movie title contained in the utilitymatrix file is used by the getplotfromomdb function to retrieve the movie's description (plot) from the website http://www.omdbapi.com/ using the requests in the Python module. The descriptions (and titles) of the movies are then saved in a CSV file (outputplots) together with the corresponding utility matrix (outputumatrix).

The other command will take the movie's descriptions and create an information retrieval system (Term Frequency, Inverse Document Frequency (TF-IDF) model) to allow the user to find movies typing some relevant words. This tf-idf model is then saved in the Django cache together with the initial recommendation systems models (CF item-based and log-likelihood ratio). The code is as follows:

```python
from django.core.management.base import BaseCommand
import os
import optparse
import numpy as np
import pandas as pd
import math
import json
import copy
from BeautifulSoup import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()
#nltk.download('stopwords')
stoplist = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from books_recsys_app.models import MovieData
from django.core.cache import cache

class Command(BaseCommand):

    option_list = BaseCommand.option_list + (
            optparse.make_option('-i', '--input', dest='input',
                                 type='string', action='store',
                                 help=('Input plots file')),
            optparse.make_option('--nmaxwords', '--nmaxwords', dest='nmaxwords',
                                 type='int', action='store',
                                 help=('nmaxwords')),
            optparse.make_option('--umatrixfile', '--umatrixfile', dest='umatrixfile',
                                 type='string', action='store',
                                 help=('umatrixfile')), 
        )
        
    def PreprocessTfidf(self,texts,stoplist=[],stem=False):
        newtexts = []
        for i in xrange(len(texts)):
            text = texts[i]
            if stem:
               tmp = [w for w in tknzr.tokenize(text) if w not in stoplist]
            else:
               tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
            newtexts.append(' '.join(tmp))
        return newtexts
    
    def handle(self, *args, **options):
        input_file = options['input']
        
        df = pd.read_csv(input_file)
        tot_textplots = df['plot'].tolist()
        tot_titles = df['title'].tolist()
        nmaxwords=options['nmaxwords']
        vectorizer = TfidfVectorizer(min_df=0,max_features=nmaxwords)
        processed_plots = self.PreprocessTfidf(tot_textplots,stoplist,True)
        mod_tfidf = vectorizer.fit(processed_plots)
        vec_tfidf = mod_tfidf.transform(processed_plots)
        ndims = len(mod_tfidf.get_feature_names())
        nmovies = len(tot_titles[:])
        
        #delete all data
        MovieData.objects.all().delete()
        
        matr = np.empty([1,ndims])
        titles = []
        cnt=0
        for m in xrange(nmovies):
            moviedata = MovieData()
            moviedata.title=tot_titles[m]
            moviedata.description=tot_textplots[m]
            moviedata.ndim= ndims
            moviedata.array=json.dumps(vec_tfidf[m].toarray()[0].tolist())
            moviedata.save()
            newrow = moviedata.array
            if cnt==0:
                matr[0]=newrow
            else:
                matr = np.vstack([matr, newrow])
            titles.append(moviedata.title)
            cnt+=1
        #cached
        cache.set('data', matr)
        cache.set('titles', titles)
        cache.set('model',mod_tfidf)

        
        #load the utility matrix
        umatrixfile = options['umatrixfile']
        df_umatrix = pd.read_csv(umatrixfile)
        Umatrix = df_umatrix.values[:,1:]
        cache.set('umatrix',Umatrix)
        #load rec methods... 
        cf_itembased = CF_itembased(Umatrix)
        cache.set('cf_itembased',cf_itembased)
        llr = LogLikelihood(Umatrix,titles)
        cache.set('loglikelihood',llr)
        
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def sim(x,y,metric='cos'):
    if metric == 'cos':
       return 1.-cosine(x,y)
    else:#correlation
       return pearsonr(x,y)[0]
       
class CF_itembased(object):
...        
class LogLikelihood(object):
...
```
To run the command the syntax is:
```python
python manage.py load_data --input=plots.csv --nmaxwords=30000  --umatrixfile=umatrix.csv
```
The input parameter takes the movie's descriptions obtained using the get_plotsfromtitles command and creates a tf-idf model (see Chapter 4, Web-mining techniques) using a maximum of words specified by the nmaxwords parameter. The data of each movie is also saved in a MovieData object (title, tf-idf representation, description, and ndim number of words of the tf-idf vocabulary). Note that the first time the command is run the stopwords from nltk.download('stopwords') (commented in the preceding code) need to be downloaded.

The tf-idf model, the title's list, and the matrix of the tf-idf movies' representations, are saved in the Django cache using the commands:
```python
from django.core.cache import cache
...
cache.set('model',mod_tfidf)
cache.set('data', matr)
cache.set('titles', titles)
```
NOTE
Note that the cache Django module (django.core.cache) needs to be loaded (at the beginning of the file) to be used.

In the same way, the utility matrix (umatrixfile parameter) is used to initialize the two recommendation systems used by the application: item-based collaborative filtering and log-likelihood ratio method. Both methods are not written in the preceding code because they are essentially the same as the code described in Chapter 5, Recommendation systems (the full code can be seen in the chapter_7 folder of the author's GitHub repository as usual). The methods and the utility matrix are then loaded into the Django cache ready to use:
```python
cache.set('umatrix',Umatrix)
   cache.set('cf_itembased',cf_itembased)
   cache.set('loglikelihood',llr)
```
Now the data (and models) can be used in the web pages just by calling the corresponding name, as we will see in the following sections.

# User sign up login/logout implementation

This application can recommend movies to different users that are registered on the website. To manage the registration process, we use the standard User Django module as we have seen in the Models sections. Each page of the website refers to the base.html page, which implements a top bar that allows the user to register or sign in (right side):

User sign up login/logout implementation

Clicking on one of the two buttons sign in or sign up will activate the code:
```python
                <form class="navbar-search pull-right" action="{% url 'auth' %}" method="GET">
                  {% csrf_token %}
                   <div style="overflow: hidden; padding-right: .5em;">
                     <input type="submit" name="auth_method" value="sign up" size="30" style="float: right" />
                     <input type="submit" name="auth_method" value="sign in" size="30" style="float: right" />
                    </div>
                </form>
                ```
The two methods refer to the urls.py:
```python
    url(r'^auth/', 'books_recsys_app.views.auth', name='auth')
```
This calls the auth function in the views.py:
```python
def auth(request):
    if request.method == 'GET':
        data = request.GET
        auth_method = data.get('auth_method')
        if auth_method=='sign in':
           return render_to_response(
               'books_recsys_app/signin.html', RequestContext(request, {})) 
        else:    
            return render_to_response(
                'books_recsys_app/createuser.html', RequestContext(request, {}))
    elif request.method == 'POST':
        post_data = request.POST
        name = post_data.get('name', None)
        pwd = post_data.get('pwd', None)
        pwd1 = post_data.get('pwd1', None)
        create = post_data.get('create', None)#hidden input
        if name and pwd and create:
           if User.objects.filter(username=name).exists() or pwd!=pwd1:
               return render_to_response(
                   'books_recsys_app/userexistsorproblem.html', RequestContext(request))
           user = User.objects.create_user(username=name,password=pwd)
           uprofile = UserProfile()
           uprofile.user = user
           uprofile.name = user.username
           uprofile.save(create=True)

           user = authenticate(username=name, password=pwd)
           login(request, user)
           return render_to_response(
               'books_recsys_app/home.html', RequestContext(request))
        elif name and pwd:
            user = authenticate(username=name, password=pwd)
            if user:
                login(request, user)
                return render_to_response(
                    'books_recsys_app/home.html', RequestContext(request))
            else:
                #notfound
                return render_to_response(
                    'books_recsys_app/nopersonfound.html', 
                       RequestContext(request))
```
The function will redirect to the sign up page as shown in the following screenshot:

User sign up login/logout implementation
If you have already registered, it will take you to the sign in page as shown in the following screenshot:

User sign up login/logout implementation
The page allows the user to create a username and password and log in to the website. The data is then used to create a new object of the User Django model and the related UserProfile object (note that the create argument is True to save the object without associating an array of rated movies):
```python
user = User.objects.create_user(username=name,password=pwd)
uprofile = UserProfile()
uprofile.user = user
uprofile.save(create=True)
user = authenticate(username=name, password=pwd)
```
The user is then logged in using the standard Django methods:
```python
from django.contrib.auth import authenticate, login
...
login(request, user)
Hence, the website top bar looks like (username:a) as shown in the following
```
screenshot:

User sign up login/logout implementation
Note that in cases where a user with the same name already exists (new sign up exception event) or where a user is not found (sign in exception event), both are implemented and the reader can look into the code to understand how these events are handled.

The sign out button refers to the urls.py:
```python
url(r'^signout/','books_recsys_app.views.signout',name='signout')
```
This calls the signout function from views.py:
```python
from django.contrib.auth import logout
…
def signout(request):
    logout(request)
    return render_to_response(
        'books_recsys_app/home.html', RequestContext(request))  
        ```
The function uses the standard Django logout method and redirects to the home page (the sign in and sign out buttons will be shown again in the top bar). The user can now search for movies to rate using the information retrieval system (search engine) described in the next section.

Information retrieval system (movies query)
In order to rate movies, the user needs to search for them using the home page:

Information retrieval system (movies query)
By Typing some relevant words in the text box, the page will call (through the urls.py corresponding home URL) the home function in the views.py file:
```python
def home(request):
    context={}
    if request.method == 'POST':
        post_data = request.POST
        data = {}
        data = post_data.get('data', None)
        if data:
            return redirect('%s?%s' % (reverse('books_recsys_app.views.home'),
                                urllib.urlencode({'q': data})))
    elif request.method == 'GET':
        get_data = request.GET
        data = get_data.get('q',None)
        titles = cache.get('titles')
        if titles==None:
            print 'load data...'
            texts = []
            mobjs = MovieData.objects.all()
            ndim = mobjs[0].ndim
            matr = np.empty([1,ndim])
            titles_list = []
            cnt=0
            for obj in mobjs[:]:
                texts.append(obj.description)
                newrow = np.array(obj.array)
                #print 'enw:',newrow
                if cnt==0:
                    matr[0]=newrow
                else:
                    matr = np.vstack([matr, newrow])
                titles_list.append(obj.title)
                cnt+=1
            vectorizer = TfidfVectorizer(min_df=1,max_features=ndim) 
            processedtexts = PreprocessTfidf(texts,stoplist,True)
            model = vectorizer.fit(processedtexts)
            cache.set('model',model)
            #cache.set('processedtexts',processedtexts)
            cache.set('data', matr)
            cache.set('titles', titles_list)
        else:
            print 'loaded',str(len(titles))
          
        Umatrix = cache.get('umatrix')
        if Umatrix==None:
            df_umatrix = pd.read_csv(umatrixpath)
            Umatrix = df_umatrix.values[:,1:]
            cache.set('umatrix',Umatrix)
            cf_itembased = CF_itembased(Umatrix)
            cache.set('cf_itembased',cf_itembased)
            cache.set('loglikelihood',LogLikelihood(Umatrix,movieslist))
            
        if not data:
            return render_to_response(
                'books_recsys_app/home.html', RequestContext(request, context))
        
        
        #load all movies vectors/titles
        matr = cache.get('data')
        titles = cache.get('titles')
        model_tfidf = cache.get('model')
        #find movies similar to the query
        queryvec = model_tfidf.transform([data.lower().encode('ascii','ignore')]).toarray()     
        sims= cosine_similarity(queryvec,matr)[0]
        indxs_sims = list(sims.argsort()[::-1])
        titles_query = list(np.array(titles)[indxs_sims][:nmoviesperquery])
        
        context['movies']= zip(titles_query,indxs_sims[:nmoviesperquery])
        context['rates']=[1,2,3,4,5]
        return render_to_response(
            'books_recsys_app/query_results.html', 
              RequestContext(request, context))
              ```
The data parameter at the beginning of the function will store the typed query and the function will use it to transform it to a vector tf-idf representation using the model already loaded in memory by the load_data command:
```python
        matr = cache.get('data')
        titles = cache.get('titles')
        model_tfidf = cache.get('model')
        ```
Also the matrix (key: matr) and the movies' titles (key: titles) are retrieved from the cache to return the list of movies similar to the query vector (see Chapter 4, Web-mining techniques for further details). Also note that in case the cache is empty, the models (and the other data) are created and loaded in memory directly from the first call of this function. For example, we can type war as a query and the website will return the most similar movies to this query (query_results.html):

Information retrieval system (movies query)
As we can see, we have five movies (at the beginning of the views.py file we can set the number of movies per query parameter: nmoviesperquery) and most of them are related to war. From this page we can rate the movies as we discuss in the following section.

# Rating system
Each user (when logged in) can rate movies simply by clicking on the rate value (1 to 5) at the side of the movie title in the movies' results page (see preceding screenshot). This action will trigger the rate_movie function in the views.py file (through the corresponding URL in urls.py):
```python
def rate_movie(request):
    data = request.GET
    rate = data.get("vote")
    movies,moviesindxs = zip(*literal_eval(data.get("movies")))
    movie = data.get("movie")
    movieindx = int(data.get("movieindx"))
    #save movie rate
    userprofile = None
    if request.user.is_superuser:
        return render_to_response(
            'books_recsys_app/superusersignin.html', RequestContext(request))
    elif request.user.is_authenticated() :
        userprofile = UserProfile.objects.get(user=request.user)
    else:
        return render_to_response(
            'books_recsys_app/pleasesignin.html', RequestContext(request))
    
    if MovieRated.objects.filter(movie=movie).filter(user=userprofile).exists():
        mr = MovieRated.objects.get(movie=movie,user=userprofile)
        mr.value = int(rate)
        mr.save()
    else:
        mr = MovieRated()
        mr.user = userprofile
        mr.value = int(rate)
        mr.movie = movie
        mr.movieindx = movieindx
        mr.save()
        
    userprofile.save()
    #get back the remaining movies
    movies = RemoveFromList(movies,movie)
    moviesindxs = RemoveFromList(moviesindxs,movieindx)
    print movies
    context = {}
    context["movies"] = zip(movies,moviesindxs)
    context["rates"] = [1,2,3,4,5]
    return render_to_response(
        'books_recsys_app/query_results.html', 
          RequestContext(request, context))
          ```
The function will store the rate of the movie in an object of the MovieRated model, and the corresponding movies rate vector of the user is updated (through the userprofile.save()). The movies not rated are then sent back to the page query_results.html. Note that the user needs to be logged in to rate a movie or the exception event that will ask the user to sign in will be shown (page: pleasesignin.html).

# Recommendation systems
This function will use the parameters set at the beginning of the views.py file:
```python
nminimumrates=5
numrecs=5
recmethod = 'loglikelihood'
```
This defines the minimum number of movies to rate before obtaining recommendations, the number of recommendations to show to the user, and the recommendation system method respectively. To show recommendations the user can click on the Recommendations button on the top bar:

Recommendation systems
This action will trigger the movies_recs function in the views.py file (through the corresponding URL defined in the urls.py file):
```python
def movies_recs(request):
    
    userprofile = None
    if request.user.is_superuser:
        return render_to_response(
            'books_recsys_app/superusersignin.html', RequestContext(request))
    elif request.user.is_authenticated():
        userprofile = UserProfile.objects.get(user=request.user)
    else:
        return render_to_response(
            'books_recsys_app/pleasesignin.html', RequestContext(request))
    ratedmovies=userprofile.ratedmovies.all()
    context = {}
    if len(ratedmovies)<nminimumrates:
        context['nrates'] = len(ratedmovies)
        context['nminimumrates']=nminimumrates
        return render_to_response(
            'books_recsys_app/underminimum.html', RequestContext(request, context))
            
    u_vec = np.array(userprofile.array)
    Umatrix = cache.get('umatrix')
    movieslist = cache.get('titles')
    #recommendation...
    u_rec = None
    if recmethod == 'cf_userbased':
        u_rec = CF_userbased(u_vec,numrecs,Umatrix)      
    elif recmethod == 'cf_itembased':
        cf_itembased = cache.get('cf_itembased')
        if cf_itembased == None:
            cf_itembased = CF_itembased(Umatrix)
        u_rec = cf_itembased.CalcRatings(u_vec,numrecs)        
    elif recmethod == 'loglikelihood':
        llr = cache.get('loglikelihood')
        if llr == None:
            llr = LogLikelihood(Umatrix,movieslist)
        u_rec = llr.GetRecItems(u_vec,True)
    #save last recs
    userprofile.save(recsvec=u_rec)
    context['recs'] = list(np.array(movieslist)[list(u_rec)][:numrecs])
    return render_to_response(
        'books_recsys_app/recommendations.html', 
          RequestContext(request, context))
          ```
The function will retrieve the rated movies vector from the corresponding UserProfile object and it will load the recommendation system method (specified by the recmethod parameter) from cache. The recommendations are first stored in the userprofile object and then returned to the recommendations.html page. For example, using the cf_itembased method:

Recommendation systems
This is a sample result page after rating the five movies related to the word war (see preceding screenshot). The reader can play more with the parameters and the different algorithms to evaluate the differences.

# Admin interface and API
In order to manage the data of the application, the admin interface and an API point can be set. From the admin panel we can see both the movie's data, and the user registered, writing the following admin.py file:
```python
from django.contrib import admin
from books_recsys_app.models import MovieData,UserProfile
      
class MoviesAdmin(admin.ModelAdmin):
      list_display = ['title', 'description']

admin.site.register(UserProfile)
admin.site.register(MovieData,MoviesAdmin)
```
After setting the corresponding admin URL on the urls.py file:
```python
url(r'^admin/', include(admin.site.urls))
```
We should see our admin panel (at http://localhost:8000/admin/) with the two models and the data within the models resembles the fields specified in the admin.py file:

Admin interface and API
To set the API endpoint to retrieve the data for each registered user, first we need to write out serializers.py specifying which fields of the UserProfile model we want to employ:
```python
from books_recsys_app.models import UserProfile
from rest_framework import serializers

class UsersSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = UserProfile
        fields = ('name', 'arrayratedmoviesindxs','lastrecs')
        ```
        
In this case, we want to collect the ID of the movies, rated by the user, and his last recommended movie's ID. Then the API is set in the api.py file as follows:
```python
from rest_framework import generics
from rest_framework.permissions import AllowAny
from rest_framework.pagination import PageNumberPagination
from books_recsys_app.serializers import UsersSerializer
from books_recsys_app.models import UserProfile

class LargeResultsSetPagination(PageNumberPagination):
    page_size = 1000
    page_size_query_param = 'page_size'
    max_page_size = 10000
    
class UsersList(generics.ListAPIView):

    serializer_class = UsersSerializer
    permission_classes = (AllowAny,)
    pagination_class = LargeResultsSetPagination
    
    def get_queryset(self):
        query = self.request.query_params.get
        if query('name'):
           return UserProfile.objects.filter(name=query('name')) 
        else:
           return UserProfile.objects.all()
           ```
Note that a query parameter name is allowed in case we want to collect only the data for one particular user. After setting the corresponding URL in the urls.py file:
```python
url(r'^users-list/',UsersList.as_view(),name='users-list')
```
The end point can be called through the curl command using the terminal:
```python
curl -X GET localhost:8000/users-list/
```
It can also be called using the swagger interface for testing purposes (see Chapter 6, Basics of Django: a simple web framework).

# Summary

We have just shown how to build an application to recommend movies using the Django framework. You now should have some degree of confidence in how to develop a professional web application using Python and the machine-learning algorithms that power it.

In the next chapter, an additional example on a movie's web sentiment reception will give you even more understanding to efficiently write your own machine-learning web application in Python.